# Number of Non-Directed Interactions

In [1]:
from pip._internal import main as pip
try:
    import csv
except ImportError:
    pip(['install', 'csv'])
    import csv
try:
    from SPARQLWrapper import SPARQLWrapper, JSON
except  ImportError:
    pip(['install', 'sparqlwrapper'])
    from SPARQLWrapper import SPARQLWrapper, JSON
    
import pandas
    
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
NonDirected = pandas.DataFrame(columns=['NonDirected Interactions Count'])
pathwayQuery = '''
      SELECT DISTINCT (count(?entity) as ?NonDirectInt)
      WHERE {
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
        MINUS { ?entity a wp:DirectedInteraction .}
      }  
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	NonDirected = NonDirected.append({
		'NonDirected Interactions Count': result["NonDirectInt"]["value"],
	 }, ignore_index=True)

In [2]:
NonDirected

,NonDirected Interactions Count
0,3642


# Total WP Interactions

In [3]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
totalWPInts = pandas.DataFrame(columns=['Total WP Interactions'])
pathwayQuery = '''
      SELECT DISTINCT (count(?entity) as ?InteractionCount)
      WHERE {
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
      }  
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	totalWPInts = totalWPInts.append({
		'Total WP Interactions': result["InteractionCount"]["value"],
	 }, ignore_index=True)

formTotalWPInts = totalWPInts.loc[:, "Total WP Interactions"].apply(pandas.to_numeric, errors = 'ignore').sum()

In [4]:
totalWPInts

,Total WP Interactions
0,15228


# Count of Interaction Types

In [5]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntTypesCount = pandas.DataFrame(columns=['Interaction Type', 'Interaction Count'])

pathwayQuery = '''
      SELECT (substr(str(?o),41) as ?IntType) (COUNT(?o) as ?IntCount)
      WHERE {
        ?entity a ?o . 
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
      } ORDER BY DESC(?IntCount)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	IntTypesCount = IntTypesCount.append({
		'Interaction Type': result["IntType"]["value"],
        'Interaction Count': result["IntCount"]["value"],
	 }, ignore_index=True)
    
IntCountInteger = IntTypesCount.loc[:, "Interaction Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 

In [6]:
IntTypesCount

,Interaction Type,Interaction Count
0,Interaction,15228
1,DirectedInteraction,11586
2,Conversion,1362
3,Catalysis,1169
4,Inhibition,1081
5,ComplexBinding,889
6,Binding,889
7,Stimulation,721
8,TranscriptionTranslation,256


# Interaction Count with Unspecified Type

In [7]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
Nonspecific = pandas.DataFrame(columns=['NonSpecified Interaction Type Count'])

pathwayQuery = '''
      SELECT DISTINCT (count(?entity) as ?NonSpecific)
      WHERE {
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
        MINUS { 
        { ?entity a wp:DirectedInteraction .  }
          UNION
        { ?entity a wp:ComplexBinding .  }
          UNION
        { ?entity a wp:Binding .  }
        }
      }  
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	Nonspecific = Nonspecific.append({
		'NonSpecified Interaction Type Count': result["NonSpecific"]["value"],
	 }, ignore_index=True)

In [8]:
Nonspecific

,NonSpecified Interaction Type Count
0,2753


# Participants for Interactions: 

In [9]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
parts4Ints = pandas.DataFrame(columns=['Interaction', 'Interaction Type', 'Interaction Participants'])


pathwayQuery = '''

    SELECT DISTINCT (substr(str(?interaction),37 ) as ?interactionId ) (substr(str(?intType), 41 ) as ?types) (GROUP_CONCAT(DISTINCT substr(str(?partType),41), ", ") AS ?participants)  WHERE {
       ?interaction a wp:Interaction ;
           dcterms:isPartOf ?pathway ;
           a ?intType .
       ?interaction wp:participants ?participant .
       ?participant a ?partType .
       ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .

       FILTER (?partType != wp:DataNode)
       FILTER (?intType != wp:Interaction)
       FIlTER (?intType != wp:DirectedInteraction)
    } 
    LIMIT 20
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    parts4Ints = parts4Ints.append({
        'Interaction': result["interactionId"]["value"],
        'Interaction Type': result["types"]["value"],
        'Interaction Participants': result["participants"]["value"],
       
    }, ignore_index=True)

In [10]:
parts4Ints

,Interaction,Interaction Type,Interaction Participants
0,WP2853_r88152/WP/Interaction/dfebd,TranscriptionTranslation,"GeneProduct, Protein"
1,WP691_r92400/ComplexBinding/bf7b2,ComplexBinding,"Complex, GeneProduct, Protein"
2,WP3871_r94406/WP/Interaction/ided6fe93b,Conversion,Metabolite
3,WP4300_r97801/WP/Interaction/id927bdc8b,Inhibition,"GeneProduct, Protein"
4,WP691_r92400/WP/Interaction/bb8fe,Catalysis,"DirectedInteraction, GeneProduct, Interaction"
5,WP2533_r98235/WP/Interaction/e1d5a,Conversion,Metabolite
6,WP2795_r97631/ComplexBinding/de943,Binding,"Complex, GeneProduct, Protein"
7,WP4524_r103502/WP/Interaction/ide98cb2fa,Conversion,Metabolite
8,WP3931_r102153/ComplexBinding/e2c08,ComplexBinding,"Complex, GeneProduct"
9,WP4136_r103640/WP/Interaction/id65ea9c11,Catalysis,"DirectedInteraction, GeneProduct, Interaction"


# Interaction counts by participants

In [11]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntCounts4Parts = pandas.DataFrame(columns=['Interaction Participants', 'Interaction Count'])

pathwayQuery = '''

    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT ?participants (COUNT(?interaction) AS ?count) WHERE {
      SELECT ?interaction (GROUP_CONCAT(substr(str(?partType),41), ", ") AS ?participants) WHERE {
        ?interaction a wp:Interaction . 
        ?interaction wp:participants ?participant . 
        ?participant a ?partType .
        ?interaction dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
        FILTER (?partType != wp:DataNode)
      } GROUP BY ?interaction
    } GROUP BY ?participants
      ORDER BY DESC(?count)
      
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    IntCounts4Parts = IntCounts4Parts.append({
        'Interaction Participants': result["participants"]["value"],
        'Interaction Count': result["count"]["value"],
       
    }, ignore_index=True)

In [12]:
IntCounts4Parts

,Interaction Participants,Interaction Count
0,"Metabolite, Metabolite",2681
1,"GeneProduct, GeneProduct",1949
2,"GeneProduct, Protein, GeneProduct, Protein",1380
3,"GeneProduct, GeneProduct, Protein",646
4,"GeneProduct, Protein",614
5,Metabolite,523
6,GeneProduct,509
7,"GeneProduct, Protein, GeneProduct",408
8,"Metabolite, GeneProduct",402
9,Protein,337


# Identifier IDs by data source

In [13]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntIDs = pandas.DataFrame(columns=['Database Source', 'Interactions Counted'])

pathwayQuery = '''
    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT Distinct (count(?Interaction) as ?IntCount) ?database 
       WHERE {
        ?Interaction a gpml:Interaction ;
            gpml:xrefDataSource ?database .
        ?Interaction dcterms:isPartOf ?pathway .
        ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
          
       } ORDER BY DESC(?IntCount)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	IntIDs = IntIDs.append({
		'Database Source': result["database"]["value"],
        'Interactions Counted': result["IntCount"]["value"],
	 }, ignore_index=True)

In [14]:
IntIDs

,Database Source,Interactions Counted
0,Uniprot-TrEMBL,213
1,Rhea,114
2,KEGG Pathway,28
3,pato,8
4,KEGG Compound,7
5,ChEBI,6
6,KEGG Reaction,3
7,Reactome,3
8,XMetDB,2
9,WikiPathways,2


# GPML RDF Interactions WITHOUT a WP RDF equivalent

In [15]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
noWPINT = pandas.DataFrame(columns=['Interactions Counted'])

pathwayQuery = '''
    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT (COUNT(?Interaction) AS ?count) 
    WHERE {

          ?Interaction a gpml:Interaction .
          ?Interaction dcterms:isPartOf ?pathway .
          ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
          MINUS {?wpInteraction wp:isAbout ?Interaction}

        } 
          ORDER BY DESC(?count)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	noWPINT = noWPINT.append({
        'Interactions Counted': result["count"]["value"],
	 }, ignore_index=True)

In [16]:
noWPINT

,Interactions Counted
0,12283


# Coverage % for GPML Interaction with Equivalent WP RDF out of Total GPML Interactions

In [17]:
IntTable = IntCountInteger
wpInteractionType = IntTable
noWPIntTable = noWPINT.loc[:,'Interactions Counted']
noWPIntIneger = noWPIntTable.apply(pandas.to_numeric, errors = 'ignore').sum()
totalWPInt = noWPIntIneger + wpInteractionType
coverageInt = (wpInteractionType / totalWPInt) * 100
formPercentInt = str(round(coverageInt, 3)) + "% of GPML Interactions with an WP equivalent"

In [18]:
formPercentInt

'72.983% of GPML Interactions with an WP equivalent'

In [19]:
percentWoInt = 100 - coverageInt
str(round(percentWoInt, 3)) + "% of GPML Interactions without a WP equivalent"

'27.017% of GPML Interactions without a WP equivalent'

# Total GPML Interactions

In [20]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
GPMLCountTotal = pandas.DataFrame(columns=['Interactions Count'])

pathwayQuery = '''
    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT DISTINCT (COUNT(?Interaction) AS ?count) 
    WHERE {

          ?Interaction a gpml:Interaction .
          ?Interaction dcterms:isPartOf ?pathway .
          ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
          
        } 
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	GPMLCountTotal = GPMLCountTotal.append({
        'Interactions Count': result["count"]["value"],
	 }, ignore_index=True)

GPMLCountTotalInt = GPMLCountTotal.loc[:,"Interactions Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 

In [21]:
GPMLCountTotalInt

27186

# % of WP RDF Interactions that are of unspecified type

In [22]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntTotalCount = pandas.DataFrame(columns=['Interaction Count'])

pathwayQuery = '''
      SELECT DISTINCT (COUNT(?genInteraction) as ?IntCount)
      WHERE {
        ?genInteraction a wp:Interaction .
        ?genInteraction dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
      }
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	IntTotalCount = IntTotalCount.append({
        'Interaction Count': result["IntCount"]["value"],
	 }, ignore_index=True)

NonSpecificTable = Nonspecific.loc[:,"NonSpecified Interaction Type Count"]
NonSpecificInteger = NonSpecificTable.apply(pandas.to_numeric, errors = 'ignore').sum()
TotalInteractions = IntTotalCount.loc[:, "Interaction Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 
percentNonSpecific = (NonSpecificInteger / TotalInteractions) * 100
formPercentNonSpecific = str(round(percentNonSpecific,3)) + '% of WP RDF Interaction with Unspecified Interaction Type'

In [23]:
formPercentNonSpecific

'18.079% of WP RDF Interaction with Unspecified Interaction Type'

In [24]:
percentWSpecificType = 100 - percentNonSpecific
str(round(percentWSpecificType, 3)) + "% of WP RDF Interactions with a specific WP RDF Interaction type"

'81.921% of WP RDF Interactions with a specific WP RDF Interaction type'

# % of NonDirected Interactions

In [25]:
NonDirectedInteger = NonDirected.loc[:, "NonDirected Interactions Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 
percentNonDirected = (NonDirectedInteger / TotalInteractions) * 100
#check this number
formPercentNonDirected = str(round(percentNonDirected, 3)) + '% of Non-directed interaction in WP RDF'

In [26]:
formPercentNonDirected

'23.916% of Non-directed interaction in WP RDF'

In [27]:
ConversionCount = int(IntTypesCount.iloc[0]["Interaction Count"])
BindingCount = int(IntTypesCount.iloc[1]["Interaction Count"])
InteractionWPRDFCount = int(IntTypesCount.iloc[2]["Interaction Count"])
DirectedWPRDFCount = int(IntTypesCount.iloc[3]["Interaction Count"])
CatalysisCount = int(IntTypesCount.iloc[4]["Interaction Count"])
TranscrTranslaCount = int(IntTypesCount.iloc[5]["Interaction Count"])
complexCount = int(IntTypesCount.iloc[6]["Interaction Count"])
InhibitionCount = int(IntTypesCount.iloc[7]["Interaction Count"])
stimulationCount = int(IntTypesCount.iloc[8]["Interaction Count"])

ConversionCount, BindingCount, InteractionWPRDFCount, DirectedWPRDFCount, CatalysisCount, TranscrTranslaCount, complexCount, InhibitionCount, stimulationCount

(15228, 11586, 1362, 1169, 1081, 889, 889, 721, 256)

In [28]:
#noWPIntIneger without eq
#GPMLCountTotalInt GPML Total
percentComplex = (complexCount / GPMLCountTotalInt ) * 100
formpercentComplex = str(round(percentComplex, 3)) + '% of complex interactions out of all GPML Interactions'
percentComplexWPRDF = (complexCount / formTotalWPInts) * 100
formpercentComplexWPRDF = str(round(percentComplexWPRDF, 3)) + '% of complex interactions out of all WP Interactions'
formpercentComplex,  formpercentComplexWPRDF

('3.27% of complex interactions out of all GPML Interactions',
 '5.838% of complex interactions out of all WP Interactions')

In [29]:
#complexCount = int(IntTypesCount.iloc[6]["Interaction Count"])
PercentConversionCount = (ConversionCount / formTotalWPInts) * 100
formPercentConversionCount = str(round(PercentConversionCount, 3)) + '% of conversion interactions out of all WP Interactions'
PercentBindingCount = (BindingCount / formTotalWPInts) * 100
formPercentBindingCount = str(round(PercentBindingCount, 3)) + '% of binding interactions out of all WP Interactions'
formPercentConversionCount, formPercentBindingCount

('100.0% of conversion interactions out of all WP Interactions',
 '76.084% of binding interactions out of all WP Interactions')

In [30]:
totalWPInts, GPMLCountTotalInt

(  Total WP Interactions
 0                 15228, 27186)